In [10]:
import pulp as plp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [11]:
power_g = [15, 100, 32, 25, 70]
cost_g = [75, 15, 0, 42, 10]

power_d = 180

ng = len(power_g)

set_ng = range(1, ng+1)

cg = {(i): cost_g[i-1] for i in set_ng}
pg = {(i): power_g[i-1] for i in set_ng}

In [12]:
opt_model = plp.LpProblem(name="Model")

In [13]:
yg  = {(i): plp.LpVariable(cat=plp.LpContinuous, lowBound=0, upBound=pg[i], name="yg_{}".format(i)) for i in set_ng}

In [14]:
constraints = plp.LpConstraint(
                 e=plp.lpSum(yg[j] for j in set_ng),
                 sense=plp.LpConstraintEQ,
                 rhs=power_d,
                 name="constraint_eq")

In [15]:
opt_model += constraints, "balance between supply and demand"

In [16]:
objective = plp.lpSum(yg[i]*cg[i] for i in set_ng)

In [17]:
opt_model.sense = plp.LpMinimize

opt_model.setObjective(objective)

In [18]:
opt_model.solve()
#print(plp.LpStatus[opt_model.status])

1

In [19]:
opt_df = pd.DataFrame.from_dict(yg, orient="index", columns = ["variable_object"])

opt_df["solution_value"] = opt_df["variable_object"].apply(lambda item: item.varValue)
opt_df

,variable_object,solution_value
1,yg_1,0.0
2,yg_2,78.0
3,yg_3,32.0
4,yg_4,0.0
5,yg_5,70.0


In [20]:
for name, c in list(opt_model.constraints.items()):
    print(c.pi)

15.0
